<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/01_prompt_engineering/01_iterative_prompt_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Iterative Prompt Development

Here, we'll iteratively analyze and refine our prompts to generate marketing copy from a product fact sheet.

## Setup

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

dotenv_path = find_dotenv() or '/content/OPENAI_API_KEY.env'
load_dotenv(dotenv_path)

openai_api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai_api_key)

In [ ]:
def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

## Generate a marketing product description from a product fact sheet

In [ ]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture,
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities:
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [ ]:
prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

**Product Description: Mid-Century Inspired Office Chair**

Elevate your workspace with our stunning Mid-Century Inspired Office Chair, a perfect blend of style and functionality. This chair is part of a beautifully curated family of office furniture that includes filing cabinets, desks, bookcases, and meeting tables, all designed to bring a touch of elegance to any environment—be it your home office or a professional setting.

Choose from a variety of shell colors and base finishes to match your aesthetic. With options for plastic back and front upholstery (SWC-100) or full upholstery (SWC-110), you can customize your chair in 10 fabric and 6 leather choices, ensuring it complements your decor seamlessly. The base finish options include sleek stainless steel, sophisticated matte black, modern gloss white, or eye-catching chrome, allowing you to create a cohesive look.

Designed for comfort and versatility, this chair is available with or without armrests, making it suitable for variou

## Issue 1: The text is too long
- Limit the number of words/sentences/characters.

In [ ]:
prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)
print(len(response.split(" ")))

Elevate your workspace with our mid-century inspired chair, featuring customizable upholstery and base finishes. Designed for comfort and style, it offers pneumatic height adjustment and options for armrests. Perfect for home or office, this chair combines elegance with functionality. Experience Italian craftsmanship today!
43


In [ ]:
# alternative 1
prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by triple backticks.

Use at most 3 sentences.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)
print(len(response.split(" ")))

Elevate your workspace with our mid-century inspired office chair, designed for both home and business settings. Available in a variety of shell colors and base finishes, this chair offers customizable upholstery options in fabric or leather, ensuring a perfect fit for your aesthetic. With its ergonomic design, pneumatic height adjustment, and durable construction, this chair combines style and comfort for a productive environment.
63


In [ ]:
# alternative 2
prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by triple backticks.

Use at most 50 characters.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)
print(len(response.split(" ")))
# print(len(response)) ## len(chars)

Mid-century inspired office chair, customizable style.
6


## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [ ]:
prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by triple backticks.

The description is intended for furniture retailers, so should be technical in nature and focus on the materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Elevate your workspace with our mid-century inspired chair, featuring a durable cast aluminum shell and a 5-wheel plastic coated aluminum base. Choose from various upholstery options, including high-density foam for comfort. Available in multiple finishes, this chair is perfect for both home and business environments.


In [ ]:
prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by triple backticks.

The description is intended for furniture retailers, so should be technical in nature and focus on the materials the product is constructed from.

At the end of the description, in a new line, include every 7-character Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our mid-century inspired office chair, expertly crafted with a cast aluminum shell and a durable nylon coating. Choose from various upholstery options, including fabric and leather, complemented by a robust 5-wheel aluminum base. Ideal for both home and business environments, this chair combines style with functionality.

SWC-1000, SWC-1100


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [ ]:
prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by triple backticks.

The description is intended for furniture retailers, so should be technical in nature and focus on the materials the product is constructed from.

At the end of the description, in a new line, include every 7-character Product ID in the technical specification.

After the description, in a new line, include a table that gives the product's dimensions. The table should have two columns.
In the first column include the name of the dimension.
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website.
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

```html
<div>
    <h2>Product Description</h2>
    <p>Introducing our mid-century inspired office chair, a perfect blend of style and functionality, designed for both home and business environments. This chair features a robust construction with a 5-wheel plastic coated aluminum base, ensuring durability and stability. The shell is crafted from cast aluminum, enhanced with a modified nylon PA6/PA66 coating, providing a sleek and modern finish. With a shell thickness of 10 mm, it offers both strength and aesthetic appeal.</p>
    <p>The seat is made from HD36 foam, available in two densities—medium (1.8 lb/ft³) or high (2.8 lb/ft³)—to cater to your comfort preferences. Choose from a variety of upholstery options, including plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather choices, allowing you to customize the chair to fit your decor.</p>
    <p>Our chair is equipped with a pneumatic adjustment feature for easy height modification, makin

## Load Python libraries to view HTML

In [ ]:
from IPython.display import display, HTML
display(HTML(response))

Dimension,Measurement (inches)
Width,20.87”
Depth,20.08”
Height,31.50”
Seat Height,17.32”
Seat Depth,16.14”
